In [7]:
import pandas as pd
import optuna

from xgboost import  XGBClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, accuracy_score, confusion_matrix


In [5]:
X_train = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/X_train.csv", header=None)
X_test = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/X_test.csv", header=None)
y_train = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/y_train.csv", header=None)
y_test = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/y_test.csv", header=None)



print("X train ",X_train.shape)
print("X test",X_test.shape)
print("Y_Train", y_train.shape)
print("Y_test", y_test.shape)

X train  (3417, 1500)
X test (375, 1500)
Y_Train (3417, 1)
Y_test (375, 1)


In [9]:
# replace 1 with 0 and -1 with 1.

# because XGBoost excepts: ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1], got [-1  1]
y_train.replace(1, 0, inplace=True)
y_train.replace(-1, 1, inplace=True)


y_test.replace(1, 0, inplace=True)
y_test.replace(-1, 1, inplace=True)

In [10]:
def objective(trial):
    # Define hyperparameters to tune
    param = {
        'objective': 'binary:logistic',
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 1.0),
        'scale_pos_weight': 1
    }

    model = XGBClassifier(**param, use_label_encoder=False, eval_metric='logloss')

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    precision = precision_score(y_test, y_pred)

    return precision

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

print(f"Best hyperparameters: {study.best_params}")
print(f"Best precision: {study.best_value}")

[I 2024-11-30 22:54:02,949] A new study created in memory with name: no-name-f30d9da2-990c-44cc-b039-a6f6ef7e7c1f


/home/pavithra/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [22:54:04] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-11-30 22:56:07,629] Trial 0 finished with value: 0.046511627906976744 and parameters: {'max_depth': 9, 'learning_rate': 0.28400861278242023, 'n_estimators': 150, 'subsample': 0.7829637164723547, 'gamma': 0.6441736721396305}. Best is trial 0 with value: 0.046511627906976744.
/home/pavithra/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [22:56:09] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-11-30 22:57:58,117] Trial 1 finished with value: 0.078125 and parameters: {'max_depth': 7, 'learning_rate': 0.2587048337731825, 'n_estimators': 145, 'subsample': 0.8492330507799103, 'gamma': 0.44168416852096093}. Best is trial 1 with value: 0.078125.
/home/pavithra

Best hyperparameters: {'max_depth': 9, 'learning_rate': 0.27575078431012434, 'n_estimators': 69, 'subsample': 0.6785914805836307, 'gamma': 0.5737136862039166}
Best precision: 0.2


In [1]:
import mlflow
model_name = "XGBost"
model_version = 1
import dagshub
dagshub.init(repo_owner='pavipd495', repo_name='spam_or_ham_fsec', mlflow=True)

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")


# Predict on a Pandas DataFrame.
import pandas as pd


Accessing as pavipd495

Initialized MLflow to track repo "pavipd495/spam_or_ham_fsec"

Repository pavipd495/spam_or_ham_fsec initialized!

2024/12/01 22:21:38 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.17.2, required: mlflow==2.18.0)
 - cloudpickle (current: 3.0.0, required: cloudpickle==3.1.0)
 - numpy (current: 1.24.3, required: numpy==1.26.4)
 - pandas (current: 2.0.3, required: pandas==2.1.4)
 - psutil (current: 5.5.1, required: psutil==6.1.0)
 - scipy (current: 1.10.1, required: scipy==1.14.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2024/12/01 22:21:38 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.12.7`, differs from the version of Python that is currently running, `Python 3.8.10`, and may be incompatible


In [6]:
model.predict(X_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,